In [ ]:
! pip install numpy
! pip install openai==1.2.3
! pip install python-dotenv
! pip install prettytable

! pip install azure-cosmos
! pip install azure-core
! pip install aiohttp

In [2]:
from dotenv import dotenv_values
from openai import AzureOpenAI
from prettytable import PrettyTable

from azure.cosmos import CosmosClient
from azure.cosmos import PartitionKey, exceptions
from azure.cosmos import ThroughputProperties

import asyncio

In [7]:
env_name = "../myconfig.env" 
config = dotenv_values(env_name)

# Cosmos Client
cosmos_endpoint = config['cosmos_endpoint']
cosmos_key = config['cosmos_key']
database_name = config['cosmos_database']
actual_ratings_name = config['cosmos_actual_ratings']
predicted_ratings_name = config['cosmos_predicted_ratings']
product_catalog_name = config['cosmos_product_catalog']

cosmos_client = CosmosClient(cosmos_endpoint, cosmos_key)

# Collection references
database = cosmos_client.get_database_client(database_name)
product_catalog_container = database.get_container_client(product_catalog_name)
actual_ratings_container = database.get_container_client(actual_ratings_name)
predicted_ratings_container = database.get_container_client(predicted_ratings_name)

# OpenAI
openai_type = config['openai_type']
openai_key = config['openai_key']
openai_base = config['openai_endpoint']
openai_version = config['openai_version']
openai_embeddings = config['openai_embeddings_deployment']

openai_client = AzureOpenAI(
    api_key=openai_key,
    api_version=openai_version,
    azure_endpoint = openai_base
)

In [8]:
def generate_azure_openai_embeddings(text):
    
    response = openai_client.embeddings.create(
        input = text, 
        model = openai_embeddings
    )
    
    embeddings = response.data[0].embedding
    return embeddings

In [9]:
def print_vector_search_results(results):
    
    print("---------Vector Search Results: --------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["Product Id", "Name", "Price", "Similarity Score", "Rating"]

    # Add rows to the table
    for product in results:
        table.add_row([
            product['Id'],
            product['Name'],
            product['Price'],
            product['SimilarityScore'],
            product['Rating']
        ])

    # Print the table
    print(table)

In [11]:
def print_predictions_from_product_page(results):
    print("\n--------Current Page Results: ---------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["Product Id", "Name", "Price", "Rating"]

    # Add rows to the table
    for product in results:
        table.add_row([
            product['Id'],
            product['Name'],
            product['Price'],
            product['Rating']
        ])

    # Print the table
    print(table)

## Product Recommendation Functions

In [12]:
def predictions_from_current_product_page(user_id, current_product_id, num_results=4):
    """
    This function displays predicted products for this user excluding the current product.
    """
    
    # Get the predicted products for the user
    predicted_results = predicted_ratings_container.read_item(item=user_id, partition_key=user_id)
    
    # Remove all but the number of items in num_results from the results
    predicted_results['Predictions'] = predicted_results['Predictions'][:num_results]
    
    # Remove the current product from the list
    user_predicted_products = [prediction for prediction in predicted_results['Predictions']
        if prediction['ProductId'] != current_product_id]
    
    predicted_products = []

    # Look up recommended products maintaining order of predicted ratings
    for item in user_predicted_products:
        product_id = item['ProductId']
        product = product_catalog_container.read_item(item=product_id, partition_key=product_id)
        if product:
            # remove the Embedding property from the product
            product.pop('Embedding')
            predicted_products.append(product)
            predicted_products[-1]['Rating'] = item['Rating']

    predicted_products = list(predicted_products)

    return predicted_products

In [ ]:
# Test the Function above on predictions excluding the current product on page
user_id = "1"
product_id = "92" # Shaun White snowboard
num_results = 10

# Predictions excluding the current product on page
on_page_predictions = predictions_from_current_product_page(user_id, product_id, num_results)

print_predictions_from_product_page(on_page_predictions)


In [13]:
def vector_search(user_prompt, product_ids, num_results=10):
    
    prompt_embedding = generate_azure_openai_embeddings(user_prompt)
    
    results = product_catalog_container.query_items(
        query='''
            SELECT TOP @num_results 
                c.id, c.Id, c.Type, c.Brand, c.Name, c.Description, c.Price, VectorDistance(c.Embedding, @embedding) AS SimilarityScore
            FROM c WHERE ARRAY_CONTAINS(@product_ids, c.id) 
            ORDER BY VectorDistance(c.Embedding, @embedding)''',
        parameters=[
            {"name": "@product_ids", "value": product_ids},
            {"name": "@embedding", "value": prompt_embedding}, 
            {"name": "@num_results", "value": num_results}
        ],
        enable_cross_partition_query=True)
   
    return results


In [15]:
def predictions_from_vector_search(user_id, user_prompt, num_results=10):
    """ 
    This function takes a user prompt search for products and returns products that are predicted for the user. 
    """

    # Get the predicted products for the user
    predicted_products = predicted_ratings_container.read_item(item=user_id, partition_key=user_id)
    
    # Convert to a dictionary
    predicted_products = {prediction['ProductId']: prediction for prediction in predicted_products['Predictions']}
    
    # Filter criteria to include product ids from the predicted products
    product_ids = list(predicted_products.keys())

    # Call the Cosmos DB NoSQL vector search function
    results = vector_search(user_prompt, product_ids, num_results)
    
    filtered_vector_search = list(results)

    # Add the Rating field to the documents in filtered_vector_search
    for item in filtered_vector_search:
        product_id = item['id']
        if product_id in predicted_products:
            item['Rating'] = predicted_products[product_id]['Rating']

    # Remove the top vector search result. Add back after sorting by rating
    top_vector_result = filtered_vector_search.pop(0)
    
    # Sort the remaining results by rating
    sorted_vector_search = sorted(filtered_vector_search, key=lambda x: x['Rating'], reverse=True)  

    # Insert the top result at the beginning of the list
    sorted_vector_search.insert(0, top_vector_result)

    return sorted_vector_search

In [17]:
# Test the function above for filtering vector search results with predicted products

user_prompt = "I want to snowboard like an Olympic Champion" #"Snowboards used by Olympic champions"
user_id = "189"
num_results = 10

# Vector Search with Predictions
vector_search_with_predictions = predictions_from_vector_search(user_id, user_prompt, num_results)
print_vector_search_results(vector_search_with_predictions)


---------Vector Search Results: --------
+------------+------------------------------+--------+---------------------+--------------------+
| Product Id |             Name             | Price  |   Similarity Score  |       Rating       |
+------------+------------------------------+--------+---------------------+--------------------+
|     92     |  Shaun White Powder Groomer  | 449.99 | 0.43047156333434977 | 6.279876708984375  |
|     22     |    Venture 2022 Snowboard    |  499   |  0.3629531261503729 | 6.241421222686768  |
|     62     |    Shadow Black Snowboard    |  379   | 0.36706966912712613 | 5.962630271911621  |
|     53     |    Raven Swift Snowboard     |  349   |  0.3609885640374669 | 5.821575164794922  |
|     73     |   Omni-Snow Dual Snowboard   | 289.99 |  0.404109018011247  | 5.8155903816223145 |
|     60     | SummitRider Snowboard Boots  |  249   |  0.3573901028293825 | 5.6328301429748535 |
|     12     |     Powder Pro Snowboard     |  399   |  0.3807208587556782 | 